In [1]:
import numpy as np
import pandas as pd
import os
import unicodedata

## Data Loading
### focus on journals related to finance, economics, management, etc., specifically:
<ol>
  <li>from CABS file use all journals in spreadsheet “CABS 2021”</li>
  <li>from JCR files use all journals in spreadsheets: BUSINESS, BUSINESS, FINANCE, ECONOMICS, MANAGEMENT
</li>
  <li>from “Wykaz…” files use the journals from the following:
ekonomia i finanse		nauki o zarządzaniu i jakości
501		506;
Use only spreadsheet “czasopisma naukowe”
</li>
</ol>

Note that if the journal appears at least in one of the source, it has to be taken into account


In [2]:
# Specify the directory containing your Excel files
directory_path = 'source files'

# Get a list of all Excel files in the directory
excel_files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx')]

# Create dictionaries to store loaded data frames for CABS, JCR, and WYKAZ
cabs_data = {}
jcr_data = {}
wykaz_data = {}

# Load from CABS file
cabs_file = [file for file in excel_files if "CABS" in file][0]
cabs_sheet = "CABS 2021"
cabs_file_path = os.path.join(directory_path, cabs_file)
try:
    cabs_df = pd.read_excel(cabs_file_path, sheet_name=cabs_sheet)
    cabs_data[cabs_file] = cabs_df
except Exception as e:
    print(f"Error loading {cabs_file}: {e}")

# Load from JCR files
jcr_files = [file for file in excel_files if "JCR" in file]
jcr_sheets = ["BUSINESS", "BUSINESS, FINANCE", "ECONOMICS", "MANAGEMENT"]
for jcr_file in jcr_files:
    jcr_file_path = os.path.join(directory_path, jcr_file)
    try:
        jcr_df = pd.read_excel(jcr_file_path, sheet_name=jcr_sheets)
        jcr_data[jcr_file] = jcr_df
    except Exception as e:
        print(f"Error loading {jcr_file}: {e}")

# Load from files containing the word “Wykaz”
wykaz_files = [file for file in excel_files if "Wykaz" in file]
for wykaz_file in wykaz_files:
    wykaz_file_path = os.path.join(directory_path, wykaz_file)
    try:
        wykaz_df = None
        with pd.ExcelFile(wykaz_file_path) as xls:
            for sheet_name in xls.sheet_names:
                if "czasopisma" in sheet_name.lower():
                    wykaz_df = pd.read_excel(wykaz_file_path, sheet_name=sheet_name, engine='openpyxl')

                    wykaz_data[wykaz_file] = wykaz_df
                    break
        if wykaz_df is None:
            print(f"No sheet containing 'czasopisma' found in {wykaz_file}")
    except Exception as e:
        print(f"Error loading {wykaz_file}: {e}")

# Print the dictionaries
print("CABS Data:")
print(cabs_data.keys())

print("\nJCR Data:")
print(jcr_data.keys())

print("\nWYKAZ Data:")
print(wykaz_data.keys())

CABS Data:
dict_keys(['CABS Journal Ranking 2021.xlsx'])

JCR Data:
dict_keys(['JCR_2021_ALL.xlsx', 'JCR_2022_ALL.xlsx'])

WYKAZ Data:
dict_keys(['20211201_Wykaz_dyscyplin_przypisanych_do_czasopism_naukowych_i_materiałów_konferencyjnych - Dec 2021 1.xlsx', '20240105_Wykaz_czasopism_naukowych_2024_styczeń.xlsx', 'Wykaz_dyscyplin_do_czasopism_i_materiałów_konferencyjnych Feb 2021.xlsx'])


In [3]:
# Replace preceeding column names with true column names
def process_dataframes(dictionary_name, data_dict):
    for file_name, data_frame in data_dict.items():
        # Check if 'issn' is present in any column name (case-insensitive)
        has_issn_column = any('issn' in str(col).lower() for col in data_frame.columns)

        if not has_issn_column:
            # Make the first row the column names
            data_frame.columns = data_frame.iloc[0]
            data_frame = data_frame[1:]  # Exclude the first row after using it as column names

            # Reset the index after excluding the first row
            data_frame.reset_index(drop=True, inplace=True)

            # Update the dataframe in the dictionary
            data_dict[file_name] = data_frame

    print(f"Processed {dictionary_name} data frames.")

process_dataframes('WYKAZ', wykaz_data)


Processed WYKAZ data frames.


In [4]:
# Rename duplicate and inconsistent column names
def rename_duplicate_columns_in_dict(data_dict):
    for df_name, df in data_dict.items():
        column_count = {}
        new_columns = []

        for column in df.columns:
            if column not in column_count:
                column_count[column] = 1
                new_column = column
            else:
                column_count[column] += 1
                new_column = f"{column}{column_count[column]}"

            new_columns.append(new_column)

        df.columns = new_columns

        # Change 'issn.1' to 'issn2' and 'e-issn.1' to 'e-issn2'
        if 'issn.1' in df.columns:
            df.rename(columns={'issn.1': 'issn2', 'e-issn.1': 'e-issn2'}, inplace=True)

        # Change 'Punktacja' to 'Punkty'
        if 'Punktacja' in df.columns:
            df.rename(columns={'Punktacja': 'Points Jan24'}, inplace=True)

        data_dict[df_name] = df

    return data_dict

# Call the function to rename duplicate columns in all DataFrames
rename_duplicate_columns_in_dict(wykaz_data)


{'20211201_Wykaz_dyscyplin_przypisanych_do_czasopism_naukowych_i_materiałów_konferencyjnych - Dec 2021 1.xlsx':          Lp.  Unikatowy Identyfikator Czasopisma  \
 0          1                                   1   
 1          2                                   2   
 2          3                                   3   
 3          4                                   4   
 4          5                                   5   
 ...      ...                                 ...   
 32671  32672                              201488   
 32672  32673                              201489   
 32673  32674                              201490   
 32674  32675                              201491   
 32675  32676                              201492   
 
                                                  Tytuł 1       issn  \
 0                                           2D Materials  2053-1583   
 1                                              3 Biotech  2190-572X   
 2                                 

In [5]:
# Rename Punkty columns to reflect date
def rename_columns_in_wykaz_data(data_dict):
    for df_name, df in data_dict.items():
        # Check if the DataFrame is the specific one
        if df_name == "20211201_Wykaz_dyscyplin_przypisanych_do_czasopism_naukowych_i_materiałów_konferencyjnych - Dec 2021 1.xlsx":
            # Check if 'Punkty' column exists
            if 'Punkty' in df.columns:
                # Rename 'Punkty' column
                df.rename(columns={'Punkty': 'Points Dec21'}, inplace=True)
        elif df_name == "Wykaz_dyscyplin_do_czasopism_i_materiałów_konferencyjnych Feb 2021.xlsx":
            # Check if 'Punkty' column exists
            if 'Punkty' in df.columns:
                # Rename 'Punkty' column
                df.rename(columns={'Punkty': 'Points Feb21'}, inplace=True)

    return data_dict

wykaz_data = rename_columns_in_wykaz_data(wykaz_data)

### Wykaz Files
Extracting the required data to create a united dataframe out of all files

In [6]:
# Create a single DataFrame from all Wykaz files
wykaz_df = pd.DataFrame(columns=['Tytuł 1', 'issn', 'e-issn', 'Tytuł 2', 'issn2', 'e-issn2'])

# Extract data from Wykaz files for categories 501 and 506
for file_name, file_data in wykaz_data.items():
    for column in file_data.columns:
        if column in [501, 506]:
            # Find all 'Points' columns in the DataFrame
            points_columns = [col for col in file_data.columns if isinstance(col, str) and col.startswith('Points')]

            # Create the list of selected columns
            selected_columns = ['Tytuł 1', 'issn', 'e-issn', 'Tytuł 2', 'issn2', 'e-issn2'] + points_columns

            # Print the actual column names in the DataFrame
            print(f"Actual Column Names in {file_name}: {file_data.columns.tolist()}")

            selected_data = file_data[file_data[column] == 'x'][selected_columns]

            # Print the selected data to see if the columns are present
            print(f"Selected Data for {file_name}:\n{selected_data.head()}")

            wykaz_df = pd.concat([wykaz_df, selected_data])

# Deduplicate the resulting DataFrame
wykaz_df = wykaz_df.drop_duplicates()

# Replace various representations of missing values in 'issn' and 'e-issn' with NaN
missing_values = ['', '         ', 'NA', 'N/A', 'NaN', 'nan', None]
wykaz_df['issn'].replace(missing_values, np.nan, inplace=True)
wykaz_df['e-issn'].replace(missing_values, np.nan, inplace=True)

# Fill NaN values in 'Tytuł 1', 'issn', 'e-issn' with values from 'Tytuł 2', 'issn2', 'e-issn2' respectively
wykaz_df['Tytuł 1'].fillna(wykaz_df['Tytuł 2'], inplace=True)
wykaz_df['issn'].fillna(wykaz_df['issn2'], inplace=True)
wykaz_df['e-issn'].fillna(wykaz_df['e-issn2'], inplace=True)

# Drop the 'Tytuł 2', 'issn2', 'e-issn2' columns
wykaz_df = wykaz_df.drop(columns=['Tytuł 2', 'issn2', 'e-issn2'])

# Display the resulting Wykaz DataFrame
print("Wykaz DataFrame:")
wykaz_df

Actual Column Names in 20211201_Wykaz_dyscyplin_przypisanych_do_czasopism_naukowych_i_materiałów_konferencyjnych - Dec 2021 1.xlsx: ['Lp.', ' Unikatowy Identyfikator Czasopisma', 'Tytuł 1', 'issn', 'e-issn', 'Tytuł 2', 'issn2', 'e-issn2', 'Points Dec21', 101, 102, 103, 104, 105, 106, 107, 201, 202, 203, 204, 205, 206, 207, 208, 209, 301, 302, 303, 304, 401, 402, 403, 404, 405, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 601, 602, 603, 604, 605, 606, 607, 701]
Selected Data for 20211201_Wykaz_dyscyplin_przypisanych_do_czasopism_naukowych_i_materiałów_konferencyjnych - Dec 2021 1.xlsx:
                                              Tytuł 1       issn     e-issn  \
9      4OR-A Quarterly Journal of Operations Research  1619-4500  1614-2411   
22  Abacus-A Journal of Accounting Finance and Bus...  0001-3072  1467-6281   
54                             Accounting and Finance  0810-5391  1467-629X   
56           Accounting Economics and Law-A Convivium  2194-6051  2152-2820   
67 

,Tytuł 1,issn,e-issn,Points Dec21,Points Jan24,Points Feb21
9,4OR-A Quarterly Journal of Operations Research,1619-4500,1614-2411,70,NaN,NaN
22,Abacus-A Journal of Accounting Finance and Bus...,0001-3072,1467-6281,70,NaN,NaN
54,Accounting and Finance,0810-5391,1467-629X,70,NaN,NaN
56,Accounting Economics and Law-A Convivium,2194-6051,2152-2820,70,NaN,NaN
67,ACCOUNTING REVIEW,0001-4826,1558-7967,200,NaN,NaN
...,...,...,...,...,...,...
31208,Internetowy Kwartalnik Antymonopolowy i Regula...,2299-8837,2299-5749,NaN,NaN,20.0
31235,Humanum. MIędzynarodowe Studia Społeczno-Human...,1898-8431,2450-0313,NaN,NaN,20.0
31259,Problemy Jakości,0137-8651,2449-9862,NaN,NaN,20.0
31277,Społeczeństwo. Edukacja. Język,2353-1266,NaN,NaN,NaN,40.0


In [7]:
# Deduplicate wykaz_df based on 'Tytuł 1', 'issn', 'e-issn'
wykaz_df['Tytuł 1'] = wykaz_df['Tytuł 1'].str.strip()
wykaz_df['Tytuł 1'] = wykaz_df['Tytuł 1'].str.encode('utf-8').str.decode('utf-8')
wykaz_df['Tytuł 1'] = wykaz_df['Tytuł 1'].apply(lambda x: unicodedata.normalize('NFKD', x))
wykaz_df['Tytuł 1'] = wykaz_df['Tytuł 1'].str.replace(r'\s+', ' ', regex=True)
wykaz_df_deduplicated = wykaz_df.groupby(['Tytuł 1', 'issn', 'e-issn']).apply(lambda group: group.ffill().bfill())
wykaz_df_deduplicated = wykaz_df.groupby(['issn', 'e-issn']).apply(lambda group: group.ffill().bfill())

# Reset index to have a clean DataFrame
wykaz_df_deduplicated.reset_index(drop=True, inplace=True)

# Drop fully duplicated rows
wykaz_df_deduplicated = wykaz_df_deduplicated.drop_duplicates(subset=['issn', 'e-issn'])

wykaz_df_deduplicated.rename(columns={'Tytuł 1': 'Title'}, inplace=True)

# Display the resulting deduplicated Wykaz DataFrame
print("Deduplicated Wykaz DataFrame:")
print(wykaz_df_deduplicated)

C:\Users\Alon\AppData\Local\Temp\ipykernel_15484\3090643752.py:6: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  wykaz_df_deduplicated = wykaz_df.groupby(['Tytuł 1', 'issn', 'e-issn']).apply(lambda group: group.ffill().bfill())
C:\Users\Alon\AppData\Local\Temp\ipykernel_15484\3090643752.py:7: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, u

Deduplicated Wykaz DataFrame:
                                                  Title       issn     e-issn  \
0        4OR-A Quarterly Journal of Operations Research  1619-4500  1614-2411   
2     Abacus-A Journal of Accounting Finance and Bus...  0001-3072  1467-6281   
4                                      Accounting Forum  0155-9982  1467-6303   
5                                Accounting and Finance  0810-5391  1467-629X   
7                                    Accounting History  1032-3732  1749-3374   
...                                                 ...        ...        ...   
8435             Journal of Tourism Management Research  2408-9117  2313-4178   
8436   Journal of Tourism Sustainability and Well-being  2795-5044  2795-5044   
8437                Research in Organizational Behavior  0191-3085  0191-3085   
8438             Recherche et Applications en Marketing  0767-3701  2051-2821   
8439            Przegląd Bezpieczeństwa Wewnętrznego  2080-1335  2720-0841  

### JCR Files
Extracting the required data to create a united dataframe out of all files

In [8]:
for file_name, year_data in jcr_data.items():
    year_suffix = file_name.split('_')[1].split('.')[0]
    year_suffix_text = f' (in {year_suffix})'

    for df_name, df in year_data.items():
        # Change column names
        df.columns = df.columns.str.replace('ISSN', 'issn', case=False)
        df.columns = df.columns.str.replace('eISSN', 'e-issn', case=False)
        df.rename(columns={'Journal name': 'Tytuł 1'}, inplace=True)

        # Add suffix to columns not mentioned
        for col in df.columns:
            if col.lower() not in ['issn', 'e-issn', 'tytuł 1', 'category']:
                df.rename(columns={col: col + year_suffix_text}, inplace=True)

In [9]:
# List of columns to include in jcr_df
columns_to_include = [
    'Tytuł 1', 'issn', 'e-issn', 'Category', '2021 JIF (in 2021)', 'JIF Quartile (in 2021)',
    '5 Year JIF (in 2021)', 'JIF Without Self Cites (in 2021)', 'JIF Percentile (in 2021)',
    '2021 JCI (in 2021)', 'JCI Percentile (in 2021)',
    '2022 JIF (in 2022)', 'JIF Quartile (in 2022)', '5 Year JIF (in 2022)',
    '5 Year JIF Quartile (in 2022)', 'JIF Rank (in 2022)', '2022 JCI (in 2022)',
    'JCI Quartile (in 2022)', 'JCI Rank (in 2022)'
]

# Create an empty DataFrame for jcr_df
jcr_df = pd.DataFrame(columns=columns_to_include)

# Concatenate DataFrames for each year
for year, year_data in jcr_data.items():
    for df_name, df in year_data.items():
        print(f"Columns for {df_name} ({year}): {df.columns}")

        # Filter columns to include only the specified ones
        existing_columns = [col for col in columns_to_include if col in df.columns]
        df_subset = df[existing_columns].copy()

        # If a column is missing, fill it with NA
        missing_columns = [col for col in columns_to_include if col not in df.columns]
        for col in missing_columns:
            df_subset[col] = np.nan

        # Concatenate to jcr_df
        jcr_df = pd.concat([jcr_df, df_subset], ignore_index=True)

# Display the resulting DataFrame
print("jcr_df:")
print(jcr_df)


Columns for BUSINESS (JCR_2021_ALL.xlsx): Index(['Tytuł 1', 'issn', 'e-issn', 'Category', 'Total Citations (in 2021)',
       '2021 JIF (in 2021)', 'JIF Quartile (in 2021)', '2021 JCI (in 2021)',
       '% of OA Gold (in 2021)', '5 Year JIF (in 2021)',
       'Immediacy Index (in 2021)', 'JIF Without Self Cites (in 2021)',
       'Eigenfactor (in 2021)', 'Normalized Eigenfactor (in 2021)',
       'Article Influence Score (in 2021)', 'JIF Percentile (in 2021)',
       'Total Articles (in 2021)', 'Citing Half-Life (in 2021)',
       'Cited Half-Life (in 2021)', 'AIS Quartile (in 2021)',
       'Citable Items (in 2021)', '% of Articles in Citable items (in 2021)',
       'JCI Percentile (in 2021)'],
      dtype='object')
Columns for BUSINESS, FINANCE (JCR_2021_ALL.xlsx): Index(['Tytuł 1', 'issn', 'e-issn', 'Category', 'Total Citations (in 2021)',
       '2021 JIF (in 2021)', 'JIF Quartile (in 2021)', '2021 JCI (in 2021)',
       '% of OA Gold (in 2021)', '5 Year JIF (in 2021)',
       'Im

In [10]:
# Find duplicate rows based on 'issn' and 'Category'
duplicate_condition = jcr_df.duplicated(subset=['issn', 'e-issn', 'Category'], keep=False)

# Extract rows with duplicates
duplicates_df_jcr = jcr_df[duplicate_condition]

duplicates_df_jcr = duplicates_df_jcr.sort_values(by=['issn', 'e-issn', 'Category'])

duplicates_df_jcr = duplicates_df_jcr.reset_index(drop=True)

# Display the DataFrame with duplicate rows
print("DataFrame with duplicate rows based on 'issn' and 'Category' (sorted by 'issn' and 'Category'):")
print(duplicates_df_jcr)

DataFrame with duplicate rows based on 'issn' and 'Category' (sorted by 'issn' and 'Category'):
                                                Tytuł 1       issn     e-issn  \
0     Abacus-A Journal of Accounting Finance and Bus...  0001-3072  1467-6281   
1     Abacus-A Journal of Accounting Finance and Bus...  0001-3072  1467-6281   
2                         ACADEMY OF MANAGEMENT JOURNAL  0001-4273  1948-0989   
3                         ACADEMY OF MANAGEMENT JOURNAL  0001-4273  1948-0989   
4                         ACADEMY OF MANAGEMENT JOURNAL  0001-4273  1948-0989   
...                                                 ...        ...        ...   
2891                  Business Strategy and Development        NaN  2572-3170   
2892                         Istanbul Business Research        NaN  2630-5488   
2893                         Istanbul Business Research        NaN  2630-5488   
2894                          Applied Economic Analysis        NaN  2632-7627   
2895         

In [11]:
# Make sure all duplicates in jcr_df are pairs of 2021-2022 with corresponding NaN values

all_nan_for_all_odd_indexes = True
failed_indexes = []

# Iterate through all odd-numbered indexes
for i in range(1, len(duplicates_df_jcr), 2):
    # Get the current row
    current_row = duplicates_df_jcr.iloc[i]

    # Check if the specified columns are all NaN
    if current_row[['2021 JIF (in 2021)', 'JIF Quartile (in 2021)', '5 Year JIF (in 2021)',
                    'JIF Without Self Cites (in 2021)', 'JIF Percentile (in 2021)',
                    '2021 JCI (in 2021)', 'JCI Percentile (in 2021)']].isna().all():
        print(f"Columns are all NaN for index {i}: {current_row[['issn', 'Category']]}")
    else:
        all_nan_for_all_odd_indexes = False
        failed_indexes.append(i)

# Print a message based on whether all columns were NaN for all odd indexes
if all_nan_for_all_odd_indexes:
    print("Yes, all specified columns are NaN for all odd indexes.")
else:
    print("No, some specified columns are not all NaN for at least one odd index.")
    print("Indexes where the rule fails:", failed_indexes)


Columns are all NaN for index 1: issn                       0001-3072
Category    BUSINESS, FINANCE - SSCI
Name: 1, dtype: object
Columns are all NaN for index 3: issn              0001-4273
Category    BUSINESS - SSCI
Name: 3, dtype: object
Columns are all NaN for index 5: issn                0001-4273
Category    MANAGEMENT - SSCI
Name: 5, dtype: object
Columns are all NaN for index 7: issn                       0001-4788
Category    BUSINESS, FINANCE - SSCI
Name: 7, dtype: object
Columns are all NaN for index 9: issn                       0001-4826
Category    BUSINESS, FINANCE - SSCI
Name: 9, dtype: object
Columns are all NaN for index 11: issn               0001-6373
Category    ECONOMICS - SSCI
Name: 11, dtype: object
Columns are all NaN for index 13: issn              0001-8392
Category    BUSINESS - SSCI
Name: 13, dtype: object
Columns are all NaN for index 15: issn                0001-8392
Category    MANAGEMENT - SSCI
Name: 15, dtype: object
Columns are all NaN for index 17: 

In [12]:
# Deduplicate rows including rows with missing identifiers for issn or e-issn
# Define boolean masks for issn and e-issn duplicates
issn_duplicates_mask = jcr_df.duplicated(subset=['issn', 'Category'], keep=False)
eissn_duplicates_mask = jcr_df.duplicated(subset=['e-issn', 'Category'], keep=False)

# Combine rows for issn duplicates
jcr_df_issn_combined = jcr_df[issn_duplicates_mask].groupby(['Tytuł 1', 'issn', 'Category'], as_index=False).apply(lambda x: x.combine_first(x.shift(-1))).reset_index(drop=True)

# Combine rows for e-issn duplicates
jcr_df_eissn_combined = jcr_df[eissn_duplicates_mask].groupby(['Tytuł 1', 'e-issn', 'Category'], as_index=False).apply(lambda x: x.combine_first(x.shift(-1))).reset_index(drop=True)

# Keep unique rows
unique_rows = jcr_df[~(issn_duplicates_mask | eissn_duplicates_mask)]

# Concatenate the deduplicated and unique rows
jcr_df_combined = pd.concat([jcr_df_issn_combined, jcr_df_eissn_combined, unique_rows], ignore_index=True)

# Drop the original duplicate rows
jcr_df_combined = jcr_df_combined.drop_duplicates(subset=['Tytuł 1', 'issn', 'e-issn', 'Category'])

# Display the resulting combined DataFrame
print("Combined jcr_df:")
print(jcr_df_combined.head())


C:\Users\Alon\AppData\Local\Temp\ipykernel_15484\3447710790.py:7: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  jcr_df_issn_combined = jcr_df[issn_duplicates_mask].groupby(['Tytuł 1', 'issn', 'Category'], as_index=False).apply(lambda x: x.combine_first(x.shift(-1))).reset_index(drop=True)
C:\Users\Alon\AppData\Local\Temp\ipykernel_15484\3447710790.py:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=

Combined jcr_df:
                                             Tytuł 1       issn     e-issn  \
0                                         3C Empresa  2254-3376  2254-3376   
1  Academia-Revista Latinoamericana de Administra...  1012-8255  2056-5127   
2                       Academy of Management Annals  1941-6520  1941-6067   
3                      ACADEMY OF MANAGEMENT JOURNAL  0001-4273  1948-0989   
4                 Academy of Management Perspectives  1558-9080        NaN   

          Category  2021 JIF (in 2021) JIF Quartile (in 2021)  \
0  BUSINESS - ESCI                 NaN                    NaN   
1  BUSINESS - SSCI               1.369                     Q4   
2  BUSINESS - SSCI              19.241                     Q1   
3  BUSINESS - SSCI              10.979                     Q1   
4  BUSINESS - SSCI               8.069                     Q1   

   5 Year JIF (in 2021)  JIF Without Self Cites (in 2021)  \
0                   NaN                               NaN   
1

In [13]:
# Group by 'issn' and 'Category', and aggregate columns, keeping the first non-NaN value
deduplicated_jcr_df_combined = jcr_df_combined.groupby(['issn', 'Category']).agg(lambda x: x.dropna().iloc[0] if x.notna().any() else np.nan).reset_index()

# Display the deduplicated DataFrame
print("Deduplicated Merged DataFrame:")
print(deduplicated_jcr_df_combined)

Deduplicated Merged DataFrame:
           issn                  Category  \
0     0001-3072  BUSINESS, FINANCE - SSCI   
1     0001-4273           BUSINESS - SSCI   
2     0001-4273         MANAGEMENT - SSCI   
3     0001-4788  BUSINESS, FINANCE - SSCI   
4     0001-4826  BUSINESS, FINANCE - SSCI   
...         ...                       ...   
1492  2708-3209           BUSINESS - ESCI   
1493  2748-1956          ECONOMICS - SSCI   
1494  2831-3232          ECONOMICS - ESCI   
1495  8756-6222          ECONOMICS - SSCI   
1496  8756-9728         MANAGEMENT - SSCI   

                                                Tytuł 1     e-issn  \
0     Abacus-A Journal of Accounting Finance and Bus...  1467-6281   
1                         ACADEMY OF MANAGEMENT JOURNAL  1948-0989   
2                         ACADEMY OF MANAGEMENT JOURNAL  1948-0989   
3                      ACCOUNTING AND BUSINESS RESEARCH  2159-4260   
4                                     ACCOUNTING REVIEW  1558-7967   
...     

### CUBS File
Extracting the required data to prepare a dataframe for concatenation

In [14]:
# Directly create a dataframe from the single file in cabs_data
cabs_df = cabs_data['CABS Journal Ranking 2021.xlsx']

# Clean column names by removing trailing spaces
cabs_df.columns = cabs_df.columns.str.strip()

# Specify the columns and their corresponding new names
columns_mapping = {
    'ISSN': 'issn',
    'Field': 'Class',
    'Journal Title': 'Journal Title (CABS)',
    'Publisher Name': 'Publisher Name',
    'AJG 2015': 'AJG 2015',
    'AJG 2018': 'AJG 2018',
    'AJG 2021': 'AJG 2021'
}

# Select and rename columns if they exist in the DataFrame
cabs_df_selected = cabs_df[columns_mapping.keys()].rename(columns=columns_mapping)

# Display the resulting DataFrame
print("CABS DataFrame:")
print(cabs_df_selected.head())

CABS DataFrame:
        issn    Class                   Journal Title (CABS)  \
0  1558-7967  ACCOUNT                      Accounting Review   
1  1873-6289  ACCOUNT  Accounting, Organizations and Society   
2  1879-1980  ACCOUNT    Journal of Accounting and Economics   
3  1475-679X  ACCOUNT         Journal of Accounting Research   
4  1911-3846  ACCOUNT       Contemporary Accounting Research   

                    Publisher Name AJG 2015 AJG 2018 AJG 2021  
0  American Accounting Association       4*       4*       4*  
1                         Elsevier       4*       4*       4*  
2                         Elsevier       4*       4*       4*  
3                  Wiley-Blackwell       4*       4*       4*  
4                  Wiley-Blackwell        4        4        4  


### Join All Sources to Get a Single Output File

In [15]:
# Perform an outer join based on 'issn'
merged_df = pd.merge(deduplicated_jcr_df_combined, wykaz_df_deduplicated, left_on=['issn', 'e-issn'], right_on=['issn', 'e-issn'], how='outer')

# Display the resulting merged DataFrame
print("Merged DataFrame:")
print(merged_df)


Merged DataFrame:
           issn                  Category  \
0     0001-3072  BUSINESS, FINANCE - SSCI   
1     0001-4273           BUSINESS - SSCI   
2     0001-4273         MANAGEMENT - SSCI   
3     0001-4788  BUSINESS, FINANCE - SSCI   
4     0001-4826  BUSINESS, FINANCE - SSCI   
...         ...                       ...   
4158  2408-9117                       NaN   
4159  2795-5044                       NaN   
4160  0191-3085                       NaN   
4161  0767-3701                       NaN   
4162  2080-1335                       NaN   

                                                Tytuł 1     e-issn  \
0     Abacus-A Journal of Accounting Finance and Bus...  1467-6281   
1                         ACADEMY OF MANAGEMENT JOURNAL  1948-0989   
2                         ACADEMY OF MANAGEMENT JOURNAL  1948-0989   
3                      ACCOUNTING AND BUSINESS RESEARCH  2159-4260   
4                                     ACCOUNTING REVIEW  1558-7967   
...                  

In [16]:
# Check for duplicates based on 'issn' and 'Category'
duplicates_df = merged_df[merged_df.duplicated(subset=['issn', 'Category'], keep=False)]

# Display the rows with duplicates
print("Duplicate Rows based on 'issn' and 'Category':")
print(duplicates_df)


Duplicate Rows based on 'issn' and 'Category':
Empty DataFrame
Columns: [issn, Category, Tytuł 1, e-issn, 2021 JIF (in 2021), JIF Quartile (in 2021), 5 Year JIF (in 2021), JIF Without Self Cites (in 2021), JIF Percentile (in 2021), 2021 JCI (in 2021), JCI Percentile (in 2021), 2022 JIF (in 2022), JIF Quartile (in 2022), 5 Year JIF (in 2022), 5 Year JIF Quartile (in 2022), JIF Rank (in 2022), 2022 JCI (in 2022), JCI Quartile (in 2022), JCI Rank (in 2022), Title, Points Dec21, Points Jan24, Points Feb21]
Index: []

[0 rows x 23 columns]


In [17]:
# Merge CABS file to add classifications
merged_all_df = pd.merge(merged_df, cabs_df_selected, left_on=['issn'], right_on=['issn'], how='outer')

In [18]:
# Check for duplicates based on 'issn' and 'Category'
duplicates_df = merged_all_df[merged_all_df.duplicated(subset=['issn', 'Category'], keep=False)]

# Display the rows with duplicates
print("Duplicate Rows based on 'issn' and 'Category':")
print(duplicates_df)

Duplicate Rows based on 'issn' and 'Category':
Empty DataFrame
Columns: [issn, Category, Tytuł 1, e-issn, 2021 JIF (in 2021), JIF Quartile (in 2021), 5 Year JIF (in 2021), JIF Without Self Cites (in 2021), JIF Percentile (in 2021), 2021 JCI (in 2021), JCI Percentile (in 2021), 2022 JIF (in 2022), JIF Quartile (in 2022), 5 Year JIF (in 2022), 5 Year JIF Quartile (in 2022), JIF Rank (in 2022), 2022 JCI (in 2022), JCI Quartile (in 2022), JCI Rank (in 2022), Title, Points Dec21, Points Jan24, Points Feb21, Class, Journal Title (CABS), Publisher Name, AJG 2015, AJG 2018, AJG 2021]
Index: []

[0 rows x 29 columns]


In [19]:
# Organise columns for saving
desired_order = [
    'Category', 'Class', 'issn', 'e-issn', 'Tytuł 1', 'Journal Title (CABS)', 'Publisher Name',
    'AJG 2015', 'AJG 2018', 'AJG 2021', '2022 JIF (in 2022)', 'JIF Quartile (in 2022)', 'JIF Rank (in 2022)',
    '5 Year JIF (in 2022)', '5 Year JIF Quartile (in 2022)', '2022 JCI (in 2022)', 'JCI Rank (in 2022)',
    'JCI Quartile (in 2022)', '2021 JIF (in 2021)', 'JIF Quartile (in 2021)', '5 Year JIF (in 2021)',
    'JIF Without Self Cites (in 2021)', 'JIF Percentile (in 2021)', '2021 JCI (in 2021)', 'JCI Percentile (in 2021)',
    'Points Feb21', 'Points Dec21', 'Points Jan24'
]

# Reorder the columns
merged_all_df = merged_all_df[desired_order]

In [20]:
# Perform small cosmetic changes before saving
merged_all_df['Class'].fillna(merged_all_df['Category'], inplace=True) # Fill missing Class values from JCR Category column
merged_all_df.drop('Category', axis=1, inplace=True)  # Drop 'Category' column
merged_all_df.sort_values(by=['issn', 'Journal Title (CABS)', 'Tytuł 1'], inplace=True)  # Sort by specified columns


In [21]:
# Clean Class to iterate through it and dynamically save it to Excel
class_counts = merged_all_df['Class'].value_counts().reset_index()
class_counts.columns = ['Class', 'Observations']
print(class_counts)

# Rename and organize classes according to CABS and JCR
class_mapping = {
    'BUSINESS, FINANCE - SSCI': 'BUSINESS, FINANCE',
    'BUSINESS, FINANCE - ESCI': 'BUSINESS, FINANCE',
    'BUSINESS - SSCI': 'BUSINESS',
    'BUSINESS - ESCI': 'BUSINESS',
    'ECONOMICS - SSCI': 'ECONOMICS',
    'ECONOMICS - ESCI': 'ECONOMICS',
    'MANAGEMENT - SSCI': 'MANAGEMENT',
    'MANAGEMENT - ESCI': 'MANAGEMENT',
    'ECON': 'CABS ECON',
    'ETHICS-CSR-MAN': 'CABS ETHICS-CSR-MAN',
    'FINANCE': 'CABS FINANCE',
    'SECTOR': 'CABS SECTOR',
    'INFO MAN': 'CABS INFO MAN',
    'ACCOUNT': 'CABS ACCOUNT',
    'SOC SCI': 'CABS SOC SCI',
    'MKT': 'CABS MKT',
    'OR&MANSCI': 'CABS OR&MANSCI',
    'OPS&TECH': 'CABS OPS&TECH',
    'PSYCH (GENERAL)': 'CABS PSYCH (GENERAL)',
    'PSYCH (WOP-OB)': 'CABS PSYCH (WOP-OB)',
    'HRM&EMP': 'CABS HRM&EMP',
    'PUB SEC': 'CABS PUB SEC',
    'IB&AREA': 'CABS IB&AREA',
    'MDEV&EDU': 'CABS MDEV&EDU',
    'INNOV': 'CABS INNOV',
    'ORG STUD': 'CABS ORG STUD',
    'ENT-SBM': 'CABS ENT-SBM',
    'REGIONAL STUDIES, PLANNING AND ENVIRONMENT': 'CABS REGIONAL STUDIES, PLANNING AND ENVIRONMENT',
    'BUS HIST & ECON HIST': 'CABS BUS HIST & ECON HIST',
    'STRAT': 'CABS STRAT'
}

merged_all_df['Class'] = merged_all_df['Class'].replace(class_mapping)

                                         Class  Observations
0                             ECONOMICS - SSCI           367
1                                         ECON           334
2                            MANAGEMENT - SSCI           216
3                             ECONOMICS - ESCI           185
4                            MANAGEMENT - ESCI           146
5                              BUSINESS - SSCI           144
6                               ETHICS-CSR-MAN           122
7                              BUSINESS - ESCI           121
8                                      FINANCE           114
9                                       SECTOR           108
10                    BUSINESS, FINANCE - SSCI           105
11                                    INFO MAN           100
12                    BUSINESS, FINANCE - ESCI            95
13                                     ACCOUNT            91
14                                     SOC SCI            88
15                      

In [22]:
# Save to Excel with separate sheets for each Class
excel_output_path = 'output file.xlsx'

# Create an ExcelWriter
with pd.ExcelWriter(excel_output_path, engine='xlsxwriter') as writer:
    # Get unique classes and handle NaN
    unique_classes = merged_all_df['Class'].fillna('unclassified').unique()

    for class_name in unique_classes:
        # Limit the length of the class name
        sheet_name = 'unclassified' if pd.isna(class_name) else str(class_name)[:31]

        # Filter rows for each class
        class_df = merged_all_df[merged_all_df['Class'].fillna('unclassified') == class_name]

        # Write to Excel sheet
        class_df.to_excel(writer, sheet_name=sheet_name, index=False)